# Data Exploration

Imports

In [1]:
import findspark
findspark.init()

from pathlib import Path

import numpy
import pandas
import re

from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext()
spark = SparkSession.builder.getOrCreate()

In [3]:
DATA_DIR = Path("/home/adityas/Kaggle/SensorWeb/data/broadcast")

cpu_rdd = sc.textFile(str(DATA_DIR/"cpu_*.csv"))
cpu_rdd = cpu_rdd.map(lambda x: x.split(","))
print(cpu_rdd.first())

['2018-02-12 01:00:05+00:00', '480.33333333209157', '0', '0', '0.5', '0', '10.0', '5.5', '0.0', '480.33333333209157', '0', '0', '0.3333333333334849', '0', '9.5', '6.0', '0.0', '480.33333333209157', '0', '0', '1.0', '0', '8.83333333331393', '8.0', '0.0', '480.66666666790843', '0', '0', '0.0', '0', '7.666666666686069', '7.333333333313931', '0.0', '480.0', '0', '0', '0.0', '0', '9.66666666668607', '5.666666666656965', '0.0', '477.0', '0', '0', '0.0', '0', '11.33333333331393', '6.0', '0.0', '480.0', '0', '0', '0.0', '0', '11.0', '5.5', '0.0', '480.5', '0', '0', '0.0', '0', '12.16666666668607', '3.6666666666569654', '0.0', '480.83333333209157', '0', '0', '0.0', '0', '11.33333333331393', '4.833333333343035', '0.0', '479.33333333209157', '0', '0', '0.0', '0', '8.83333333331393', '8.33333333331393', '0.0', '480.5', '0', '0', '0.6666666666669698', '0', '9.83333333331393', '8.0', '0.0', '479.5', '0', '0', '0.0', '0', '12.16666666668607', '6.0', '0.0']


Define function to read column names from text file.

In [4]:
def get_columns(filename):
    
    columns = []
    
    with open(filename,"r") as f:
        
        for line in f.readlines():
            columns.append(re.sub(r"\(|\)|\,|\.","_",re.sub(r"'|\s","",line.strip())))
            
    return columns

In [5]:
CPU_COLUMNS = DATA_DIR/"cpu_columns.txt"

columns = ["Time"] + get_columns(str(CPU_COLUMNS))


In [6]:
#df = spark.createDataFrame(cpu_rdd, schema)
df = cpu_rdd.toDF(columns)

In [7]:
print(dir(df))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_jcols', '_jdf', '_jmap', '_jseq', '_lazy_rdd', '_sc', '_schema', '_sort_cols', 'agg', 'alias', 'approxQuantile', 'cache', 'checkpoint', 'coalesce', 'collect', 'columns', 'corr', 'count', 'cov', 'createGlobalTempView', 'createOrReplaceTempView', 'createTempView', 'crossJoin', 'crosstab', 'cube', 'describe', 'distinct', 'drop', 'dropDuplicates', 'drop_duplicates', 'dropna', 'dtypes', 'explain', 'fillna', 'filter', 'first', 'foreach', 'foreachPartition', 'freqItems', 'groupBy', 'groupby', 'head', 'intersect', 'isLocal', 'isStreaming', 'is_cached', 'join', 'limit', 'na', 'orderBy', 'persist', 'printSchema', 'randomSplit

In [8]:
df.columns

['Time',
 '_cpu_value___host_bb1_localdomain___type_instance_idle___',
 '_cpu_value___host_bb1_localdomain___type_instance_interrupt___',
 '_cpu_value___host_bb1_localdomain___type_instance_nice___',
 '_cpu_value___host_bb1_localdomain___type_instance_softirq___',
 '_cpu_value___host_bb1_localdomain___type_instance_steal___',
 '_cpu_value___host_bb1_localdomain___type_instance_system___',
 '_cpu_value___host_bb1_localdomain___type_instance_user___',
 '_cpu_value___host_bb1_localdomain___type_instance_wait___',
 '_cpu_value___host_bb10_localdomain___type_instance_idle___',
 '_cpu_value___host_bb10_localdomain___type_instance_interrupt___',
 '_cpu_value___host_bb10_localdomain___type_instance_nice___',
 '_cpu_value___host_bb10_localdomain___type_instance_softirq___',
 '_cpu_value___host_bb10_localdomain___type_instance_steal___',
 '_cpu_value___host_bb10_localdomain___type_instance_system___',
 '_cpu_value___host_bb10_localdomain___type_instance_user___',
 '_cpu_value___host_bb10_localdo

In [9]:
for column in df.columns[1:]:
    df = df.withColumn(column, df[column].cast("float"))

In [10]:
df.dtypes

[('Time', 'string'),
 ('_cpu_value___host_bb1_localdomain___type_instance_idle___', 'float'),
 ('_cpu_value___host_bb1_localdomain___type_instance_interrupt___', 'float'),
 ('_cpu_value___host_bb1_localdomain___type_instance_nice___', 'float'),
 ('_cpu_value___host_bb1_localdomain___type_instance_softirq___', 'float'),
 ('_cpu_value___host_bb1_localdomain___type_instance_steal___', 'float'),
 ('_cpu_value___host_bb1_localdomain___type_instance_system___', 'float'),
 ('_cpu_value___host_bb1_localdomain___type_instance_user___', 'float'),
 ('_cpu_value___host_bb1_localdomain___type_instance_wait___', 'float'),
 ('_cpu_value___host_bb10_localdomain___type_instance_idle___', 'float'),
 ('_cpu_value___host_bb10_localdomain___type_instance_interrupt___', 'float'),
 ('_cpu_value___host_bb10_localdomain___type_instance_nice___', 'float'),
 ('_cpu_value___host_bb10_localdomain___type_instance_softirq___', 'float'),
 ('_cpu_value___host_bb10_localdomain___type_instance_steal___', 'float'),
 ('_c